# Step 1: Create the SparkSession Object

Well explained in this blog: [Linear_Regression](https://bayangp0.wixsite.com/2scr/blog/linear_regression)

In [38]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("lin_reg").getOrCreate()

In [4]:
spark

# Step 2: Read the Dataset

In [5]:
df = spark.read.csv("data/Linear_regression_dataset.csv", inferSchema=True, header=True)

# Step 3: Exploratory Data Analysis

In [6]:
df

DataFrame[var_1: int, var_2: int, var_3: int, var_4: double, var_5: double, output: double]

In [7]:
print((df.count(), len(df.columns)))

(1232, 6)


In [8]:
df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)



In [10]:
df.describe().show(5,False)

+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|var_1            |var_2            |var_3             |var_4               |var_5               |output             |
+-------+-----------------+-----------------+------------------+--------------------+--------------------+-------------------+
|count  |1232             |1232             |1232              |1232                |1232                |1232               |
|mean   |715.0819805194806|715.0819805194806|80.90422077922078 |0.3263311688311693  |0.25927272727272715 |0.39734172077922014|
|stddev |91.5342940441652 |93.07993263118064|11.458139049993724|0.015012772334166148|0.012907228928000298|0.03326689862173776|
|min    |463              |472              |40                |0.277               |0.214               |0.301              |
|max    |1009             |1103             |116               |0.373               |0.294               |0.491

In [11]:
df.head(3)

[Row(var_1=734, var_2=688, var_3=81, var_4=0.328, var_5=0.259, output=0.418),
 Row(var_1=700, var_2=600, var_3=94, var_4=0.32, var_5=0.247, output=0.389),
 Row(var_1=712, var_2=705, var_3=93, var_4=0.311, var_5=0.247, output=0.417)]

In [12]:
from pyspark.sql.functions import corr

In [13]:
df.select(corr('var_1','output')).show()

+-------------------+
|corr(var_1, output)|
+-------------------+
| 0.9187399607627283|
+-------------------+



# Step 4: Feature Engineering

In [14]:
from pyspark.ml.linalg import Vector

In [15]:
from pyspark.ml.feature import VectorAssembler

In [16]:
df.columns

['var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'output']

In [17]:
vec_assmebler=VectorAssembler(inputCols=['var_1', 'var_2', 'var_3', 'var_4', 'var_5'],outputCol='features')

In [18]:
features_df=vec_assmebler.transform(df)

In [19]:
features_df.printSchema()

root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- output: double (nullable = true)
 |-- features: vector (nullable = true)



In [20]:
features_df.select('features').show(5,False)

+------------------------------+
|features                      |
+------------------------------+
|[734.0,688.0,81.0,0.328,0.259]|
|[700.0,600.0,94.0,0.32,0.247] |
|[712.0,705.0,93.0,0.311,0.247]|
|[734.0,806.0,69.0,0.315,0.26] |
|[613.0,759.0,61.0,0.302,0.24] |
+------------------------------+
only showing top 5 rows



In [21]:
model_df=features_df.select('features','output')

In [22]:
model_df.show(5,False)

+------------------------------+------+
|features                      |output|
+------------------------------+------+
|[734.0,688.0,81.0,0.328,0.259]|0.418 |
|[700.0,600.0,94.0,0.32,0.247] |0.389 |
|[712.0,705.0,93.0,0.311,0.247]|0.417 |
|[734.0,806.0,69.0,0.315,0.26] |0.415 |
|[613.0,759.0,61.0,0.302,0.24] |0.378 |
+------------------------------+------+
only showing top 5 rows



In [23]:
print((model_df.count(), len(model_df.columns)))

(1232, 2)


# Step 5: Splitting the Dataset

In [25]:
train_df,test_df=model_df.randomSplit([0.7,0.3])

In [26]:
print((train_df.count(), len(train_df.columns)))

(872, 2)


In [27]:
print((test_df.count(), len(test_df.columns)))

(360, 2)


# Step 6: Build and Train Linear Regression Model

In [28]:
from pyspark.ml.regression import LinearRegression

In [29]:
lin_Reg=LinearRegression(labelCol='output')

In [30]:
lr_model=lin_Reg.fit(train_df)

In [31]:
print(lr_model.coefficients)

[0.0003452204574077164,4.8217123559315986e-05,0.0001163662863995595,-0.647426757360723,0.47660872005867094]


In [32]:
print(lr_model.intercept)

0.19449634134402397


In [33]:
training_predictions=lr_model.evaluate(train_df)

In [34]:
print(training_predictions.r2)

0.8663172713955949


# Step 7: Evaluate Linear Regression Model on Test Data

In [35]:
test_predictions=lr_model.evaluate(test_df)

In [36]:
print(test_predictions.r2)

0.8755200843664626


In [37]:
print(test_predictions.meanSquaredError)

0.00013811650213452927


# Conclusion

In this tutorial, we went over the process of building a Linear Regression
model using PySpark and also explained the process behind finding the
most optimal coefficients and intercept values.